In [2]:
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
#初始化数据
import pandas as pd
from sklearn.preprocessing import StandardScaler,MinMaxScaler

In [3]:
# 导入需要处理的数据
# 蓝莓货架期数据
data = pd.read_csv('./blueberryData0.csv')

In [4]:
col = ['day', 'hardness', 'L', 'a', 'b', 'detE', 'solubleSolid',
       'titrableAcid','weightLoss','rotRate','anthocyanin','totalPhenol','flavonoid','shelfLife']
datacol = data[col]
datacol.isnull().sum()

day             0
hardness        0
L               0
a               0
b               0
detE            0
solubleSolid    0
titrableAcid    0
weightLoss      0
rotRate         0
anthocyanin     0
totalPhenol     0
flavonoid       0
shelfLife       0
dtype: int64

In [5]:
datacol.describe()

day   hardness           L           a           b        detE  \
count  111.000000  111.00000  111.000000  111.000000  111.000000  111.000000   
mean    26.918919    1.39027   34.345045   -0.201441   -0.257387    7.943572   
std     17.254847    0.15267    3.330875    0.173477    0.032325    3.241749   
min      0.000000    1.12000   27.360000   -0.890000   -0.340000    0.000000   
25%     12.000000    1.27000   31.965000   -0.240000   -0.280000    5.776875   
50%     24.000000    1.38000   33.870000   -0.200000   -0.260000    8.380030   
75%     42.000000    1.48500   36.475000   -0.145000   -0.230000   10.285308   
max     54.000000    1.86000   43.310000    0.330000   -0.180000   14.890057   

       solubleSolid  titrableAcid  weightLoss     rotRate  anthocyanin  \
count    111.000000    111.000000  111.000000  111.000000   111.000000   
mean      10.702703      0.306144    2.251441    3.630631    11.517117   
std        0.763301      0.011321    1.449331    2.558152     0.876763   
min        8.500000      0.291000    0.000000    0.000000     9.550000   
25%       10.250000      0.299000    0.990000    2.000000    10.915000   
50%       10.800000      0.304000    2.070000    3.000000    11.590000   
75%       11.350000      0.312000    3.490000    5.000000    12.105000   
max       11.900000      0.336000    4.680000   10.000000    13.450000   

       totalPhenol   flavonoid   shelfLife  
count   111.000000  111.000000  111.000000  
mean      4.199550    3.607928    1.783784  
std       0.696995    0.641591    3.845671  
min       2.930000    2.270000    0.000000  
25%       3.720000    3.205000    0.000000  
50%       4.180000    3.580000    0.000000  
75%       4.690000    4.030000    0.000000  
max       5.890000    5.440000   12.000000

In [6]:
# 设置时间为数据的索引
datacol.index = datacol['day']
blueBerry = ['hardness', 'L', 'a', 'b', 'detE', 'solubleSolid',
       'titrableAcid','weightLoss','rotRate','anthocyanin','totalPhenol','flavonoid','shelfLife']
datag = datacol[blueBerry]
datal = datacol[blueBerry]

In [7]:
datal.head()

hardness      L     a     b      detE  solubleSolid  titrableAcid  \
day                                                                      
0        1.64  42.25 -0.19 -0.28  0.000000           9.1         0.328   
0        1.59  39.07 -0.20 -0.28  3.180016           9.4         0.335   
0        1.62  36.41 -0.24 -0.26  5.840248           9.0         0.335   
0        1.62  39.65 -0.12 -0.20  2.602172           9.1         0.335   
0        1.63  37.99 -0.28 -0.25  4.261056           9.8         0.330   

     weightLoss  rotRate  anthocyanin  totalPhenol  flavonoid  shelfLife  
day                                                                       
0           0.0        0        12.07         4.91       4.64         12  
0           0.0        0        12.79         5.66       4.76         12  
0           0.0        0        13.30         4.84       5.44         12  
0           0.0        0        13.45         5.19       4.32         12  
0           0.0        0        12.10         4.54       4.75         12

In [14]:
datal.max()

hardness         1.860000
L               43.310000
a                0.330000
b               -0.180000
detE            14.890057
solubleSolid    11.900000
titrableAcid     0.336000
weightLoss       4.680000
rotRate         10.000000
anthocyanin     13.450000
totalPhenol      5.890000
flavonoid        5.440000
shelfLife       12.000000
dtype: float64

In [16]:
# 数据逆归一化
# max_min = [datal['peel'].max(), datal['peel'].min()] # 原来的最大值最小值, 反放缩
# scaler = MinMaxScaler(feature_range=(0,1))
# sel_col = datal.columns


In [18]:
# 特征选择后的数据
feature_selection = ['totalPhenol', 'titrableAcid', 'flavonoid', 'detE', 'rotRate', 'weightLoss','hardness','shelfLife']
datal = datal[feature_selection]

In [20]:
# 定义数据
data = datal

In [22]:
def prepare_data(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        seq = data.iloc[i:i + seq_length].values  # 获取连续的seq_length行，并转换为NumPy数组
        X.append(seq)
        y.append(data.iloc[i + seq_length].values)  # 获取下一行，并转换为NumPy数组作为标签
    return np.array(X), np.array(y)

In [24]:
seq_length = 3
X, y = prepare_data(data, seq_length)

In [26]:
# 转换为 PyTorch 张量
X_tensor = torch.from_numpy(X).float()
y_tensor = torch.from_numpy(y).float()

# 使用 DataLoader 封装数据
dataset = TensorDataset(X_tensor, y_tensor)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# 添加注意力机制
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.hidden_size = hidden_size
        self.attn = nn.Linear(input_size * 2, seq_length)

    def forward(self, hidden, encoder_outputs):
        seq_len = encoder_outputs.size(1)
        # h = hidden.repeat(1, seq_len, 1)
        h = hidden
        energy = torch.tanh(self.attn(torch.cat((h, encoder_outputs), dim=2)))
        attention_weights = F.softmax(energy, dim=1)
        context_vector = torch.bmm(attention_weights.transpose(1, 2), encoder_outputs)
        return context_vector

# 定义模型
class CNN_GRU_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, kernel_size):
        super(CNN_GRU_Model, self).__init__()
        # 增加卷积层的输出通道数
        self.cnn_layer = nn.Conv1d(in_channels=input_size, out_channels=input_size, kernel_size=kernel_size)
        self.conv_output_size = input_size - kernel_size + 1  # 计算卷积层输出大小
        self.gru_layer = nn.GRU(self.conv_output_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, input_size)

        # 添加注意力机制的层
        self.attention = Attention(hidden_size)

    def forward(self, x):
        # 转置输入以适应卷积层的要求
        x = x.transpose(1, 2)  # 将最后两个维度交换位置，从(2, 3, 14)变为(2, 14, 3)

        # CNN层
        x = self.cnn_layer(x)  # 卷积层输入形状为(batch_size, 特征数, 时间步长)
        x = x.permute(0, 2, 1)  # 重新排列维度，将序列长度放在第 1 维

        # 注意力机制
        attn_weights = self.attention(x, x)
        # 将注意力权重应用到CNN的输出上
        context = torch.bmm(attn_weights.transpose(1,2), x).squeeze(1)
        # GRU层
        _, h_n = self.gru_layer(context)

        # 全连接层
        out = self.fc(h_n[-1])

        return out

# 初始化模型
input_size = X.shape[2]  # 特征数
hidden_size = 64
num_layers = 1
kernel_size = 1
model = CNN_GRU_Model(input_size, hidden_size, num_layers, kernel_size)
# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 1000
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# 预测的值
with torch.no_grad():
    last_sequence = torch.from_numpy(data[-seq_length:].values).float().unsqueeze(0)
    prediction = model(last_sequence)
    print("预测结果:", prediction.numpy())

# 保存模型
torch.save(model.state_dict(), '1cnn_gru_ae_3_fs_model.pth')


Epoch [100/1000], Loss: 1.5054
Epoch [200/1000], Loss: 0.4693
Epoch [300/1000], Loss: 0.1058
Epoch [400/1000], Loss: 0.0617
Epoch [500/1000], Loss: 0.3418
Epoch [600/1000], Loss: 0.0592
Epoch [700/1000], Loss: 0.0214
Epoch [800/1000], Loss: 0.0187
Epoch [900/1000], Loss: 0.0522
Epoch [1000/1000], Loss: 0.0326
预测结果: [[ 3.4439151  0.3163814  3.6541963 11.328713   8.410614   4.507376
   1.2179528  0.045407 ]]


In [28]:
last_sequence

tensor([[[ 4.1100,  0.3140,  3.3700, 14.6001,  6.0000,  4.5800,  1.1700,
           0.0000],
         [ 3.7300,  0.3130,  3.2200, 13.7401,  7.0000,  4.5400,  1.1700,
           0.0000],
         [ 4.0200,  0.3120,  3.2100, 11.7301,  9.0000,  4.3500,  1.1800,
           0.0000]]])